### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import joblib
import glob

from tkinter import *
import dtale as dt

import plotly.graph_objects as go
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

from sklearn import metrics
from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', '{:,.6}'.format)
# pd.set_option('max_columns', 100)

### Preparing the Dataset

In [2]:
PATH = "./data/Historic Grid Trade Master Agreement (GTMA)/gtma_trades_20*.csv"
df1 = pd.concat([pd.read_csv(g) for g in glob.glob(PATH)])
#? df1 = Orig, No formatting / feature eng'g

df1.head(6)
# df1.shape
# df1.info()


,Trade ID,Start Time,End Time,Volume,Price,Cost,SO Flag
0,ES00109856,2021-04-01T01:00:00Z,2021-04-01T02:00:00Z,-210.0,41.0,"-8,610.0",T
1,ES00109857,2021-04-01T01:00:00Z,2021-04-01T02:00:00Z,-90.0,39.28,"-3,535.2",T
2,ES00109858,2021-04-01T02:00:00Z,2021-04-01T03:00:00Z,-150.0,37.59,"-5,638.5",T
3,ES00109859,2021-04-01T02:00:00Z,2021-04-01T03:00:00Z,-100.0,37.53,"-3,753.0",T
4,ES00109860,2021-04-01T03:00:00Z,2021-04-01T04:00:00Z,-10.0,39.76,-397.6,T
5,ES00109861,2021-04-01T04:00:00Z,2021-04-01T05:00:00Z,-100.0,51.0,"-5,100.0",F


$\qquad$ Revisiting this notebook? We may use joblib easily load the edited dataframes.

In [5]:

#* Dumping the 'base' / default dataframe. 
joblib.dump(df1,"./data/GTMA Trades_df")

['./data/GTMA Trades_df']

In [2]:

df1 = joblib.load("./data/GTMA Trades_df")
# df1A = joblib.load("./data/GTMA Trades_df1A")

# df1.shape
# df1A.head(5)

### (Basic) Inspection, Exploration of DFs

In [12]:
df1.columns

Index(['Trade ID', 'Start Time', 'End Time', 'Volume', 'Price', 'Cost',
       'SO Flag'],
      dtype='object')

In [13]:
df1["Trade ID"].nunique()

114232

In [14]:
df1["Trade ID"].isnull().sum()

0

In [15]:
df1["Trade ID"].duplicated(keep=False).sum()

20

In [16]:
df1["Trade ID"].duplicated().sum()

10

In [19]:

#? Practice of Logic filtering for the rows. And desired Columns only.
df1[(df2["Price"] > 32.51) & (30.69 <= df2["Cost"] )][["Trade ID","Cost"]]

,Trade ID,Cost
10,ES00109866,1.1421e+05
274,ES00110130,2.1375e+05
275,ES00110131,1.16336e+05
276,ES00110132,1.07008e+05
710,ES00110568,1.25136e+05
...,...,...
2931,ES00007745,"99,880.0"
2932,ES00007746,"99,820.0"
2933,ES00007747,"3,600.0"
2934,ES00007748,"16,000.0"


### GUI-assisted DF Inspection & Exploration

In [ ]:

#* import dtale as dt

# df1_dt = dt.show(df1, drop_index=True, open_browser=True)
dt_df1 = dt.show(df1, drop_index=True)
dt_df1

# df1A_dt = dt.show(df1A)
# df1A_dt

### Feature Engineering

$\qquad$ Dropping duplicates, creating new features based on the trades' starting & ending time.

In [31]:
df1.fillna('0', inplace=True)
df1 = df1.query("`End Time` >= '2016'")
df1 = df1.sort_values(['Start Time'], ascending=[True])

df1.drop_duplicates(subset='Trade ID',inplace=True)


df1["Start Time_D&T"] = pd.to_datetime(pd.to_datetime(df1["Start Time"]).dt.strftime("%Y-%m-%d-%H:%M:%S"))
df1["End Time_D&T"] = pd.to_datetime(pd.to_datetime(df1["End Time"]).dt.strftime("%Y-%m-%d-%H:%M:%S"))
df1["Trade Hours Duration"] = pd.to_timedelta((df1.loc[:,"End Time_D&T"] - df1.loc[:,"Start Time_D&T"]),unit='h')

df1["Start Hour"] = pd.to_numeric((df1["Start Time_D&T"]).dt.strftime("%H"))

#? temp_durn = Temporary container for Trade Hours Duration
temp_durn = []
for i in df1["Trade Hours Duration"]:
    temp_durn.append((i.seconds)//3600)

df1.loc[:,"Trade Hours Duration"] = temp_durn


df2 = df1
drop_cols = ["Trade ID","Start Time","End Time"]
df2.drop(columns=drop_cols, inplace=True)
df2.head(4)
# df2.info()

#? df2 == w/ Feat. eng'g.


,Volume,Price,Cost,SO Flag,Start Time_D&T,End Time_D&T,Trade Hours Duration,Start Hour
2787,"1,533.0",70.0,1.0731e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
1471,"1,610.0",63.5,1.02235e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
2064,"1,715.0",72.2,1.23823e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
2080,"-2,400.0",13.63,"-32,712.0",T,2015-12-31 23:00:00,2016-01-01 09:00:00,10,23


In [32]:
joblib.dump(df2,"./data/GTMA Trades_df2")

['./data/GTMA Trades_df2']

In [2]:
df2 = joblib.load("./data/GTMA Trades_df2")

In [ ]:
dt_df2 = dt.show(df2, drop_index=True)
dt_df2

### "Manual" EDA -- By Filters

> Energy Trade Cost Paid By UK ESO

In [15]:
a_Et_By = df2[(df2["SO Flag"]=="F") & (df2["Cost"]> 0)]
a_Et_By["Cost"].sum()

171615158.875

> System Trade Cost Paid By UK ESO

In [16]:
b_St_By = df2[(df2["SO Flag"]=="T") & (df2["Cost"]> 0)]
b_St_By["Cost"].sum()

708574911.271

> Energy Trade Cost Paid To UK ESO

In [17]:
c_Et_To = df2[(df2["SO Flag"]=="F") & (df2["Cost"] < 0)]
abs(c_Et_To["Cost"].sum())

50136446.879999995

> System Trade Cost Paid To UK ESO

In [18]:
d_St_To = df2[(df2["SO Flag"]=="T") & (df2["Cost"] < 0)]
abs(d_St_To["Cost"].sum())

232090347.51635003

In [19]:
dict_costs = {
    "Energy T. Cost Paid By UK ESO" : a_Et_By["Cost"].sum(),
    "System T. Cost Paid By UK ESO" : b_St_By["Cost"].sum(),
    "Energy T. Cost Paid To UK ESO" : abs(c_Et_To["Cost"].sum()),
    "System T. Cost Paid To UK ESO" : abs(d_St_To["Cost"].sum())
        }

In [20]:
def dictionary_val_sorter(d_in: dict):
    ###?
    #? Sorts by values, Returns the sorted keys. #?
    ###?
    sorted_dict = {}
    sorted_keys = sorted(d_in, key=d_in.get,reverse=True)

    for w in sorted_keys:
        sorted_dict[w] = d_in[w]
    return sorted_dict.keys()

In [21]:
for i in dictionary_val_sorter(dict_costs):
     print(f"{i}, {dict_costs.get(i)}")

System T. Cost Paid By UK ESO, 708574911.271
System T. Cost Paid To UK ESO, 232090347.51635003
Energy T. Cost Paid By UK ESO, 171615158.875
Energy T. Cost Paid To UK ESO, 50136446.879999995


In [54]:
import plotly.graph_objects as go

pie_labels = [i for i in dictionary_val_sorter(dict_costs)]
pie_d = [dict_costs.get(i) for i in dictionary_val_sorter(dict_costs)]

colors = px.colors.qualitative.Prism[3:6]

fig = go.Figure(data=[go.Pie(labels=pie_labels, values=pie_d)])
fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=14,
                  marker=dict(colors=colors, line=dict(color='#A9A9A9', width=2)))
fig.update_layout(
    autosize=False,
    width=500,
    height=450,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
                )
                )
    # paper_bgcolor="LightSteelBlue",

fig.show()


-------------------

### EDA by Visualizations

Guide questions.

1. Around what times do the trades with high Volumes occur?
2. Something with scatterplot?
3. 


Now, let's return to dtale.

In [ ]:
dt_df2 = dt.show(df2, drop_index=True)
dt_df2

In [ ]:
-257.5  162.5

df2_noouts

In [ ]:
g = sns.relplot(x="Start Time_D", y="Cost", hue="Start Hour_Bins" , data=df1A, kind="scatter")
g

# g = sns.relplot(x=df1A[["Start Time_D",""]], y="Cost", hue="Start Hour_Bins", kind="scatter")

#? Size / dim | Cirlces
#? Per bin visualization? Or via plotly?

In [ ]:

#? Plotting Swarmplot, non-Time perspective | x = Cost, y = bins, hue = Flag
# ax = sns.swarmplot(x="total_bill", y="day", data=tips)

g_swarm_nontime = sns.swarmplot(x="Cost", y="Start Hour_Bins", hue="SO Flag",data=df1A,dodge=True, orient="h")
g_swarm_nontime

### Splitting the Dataset

In [4]:
df_sys = df2[df2["SO Flag"]=="T"]
df_sys.head(6)

,Volume,Price,Cost,SO Flag,Start Time_D&T,End Time_D&T,Trade Hours Duration,Start Hour
2787,"1,533.0",70.0,1.0731e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
1471,"1,610.0",63.5,1.02235e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
2064,"1,715.0",72.2,1.23823e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
2080,"-2,400.0",13.63,"-32,712.0",T,2015-12-31 23:00:00,2016-01-01 09:00:00,10,23
2079,"-1,200.0",13.31,"-15,972.0",T,2015-12-31 23:00:00,2016-01-01 09:00:00,10,23
2788,"1,890.0",61.0,1.1529e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23


In [5]:
df_enr = df2[df2["SO Flag"]=="F"]
df_enr.shape

(28412, 8)

In [35]:

#* Checking if splitting the data into Energy Trades & System Trades was successful.
bool(len(df2.index) == len(df_enr.index) + len(df_sys.index))

True

In [42]:
df2.head(4)

,Volume,Price,Cost,SO Flag,Start Time_D&T,End Time_D&T,Trade Hours Duration,Start Hour
2787,"1,533.0",70.0,1.0731e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
1471,"1,610.0",63.5,1.02235e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
2064,"1,715.0",72.2,1.23823e+05,T,2015-12-31 23:00:00,2016-01-01 06:00:00,7,23
2080,"-2,400.0",13.63,"-32,712.0",T,2015-12-31 23:00:00,2016-01-01 09:00:00,10,23


In [6]:
joblib.dump(df_enr,"./data/GTMA Trades_df_enr")
joblib.dump(df_sys,"./data/GTMA Trades_df_sys")

['./data/GTMA Trades_df_sys']

-----------------------------------------------------------------------------------
